In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("C:/Users/ETHIRAJ/Desktop/Recommendation_of_products/Online_Retail.csv")
print(df.head(6))

  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   
5    536365     22752         SET 7 BABUSHKA NESTING BOXES         2   

        InvoiceDate  UnitPrice  CustomerID         Country  
0  01-12-2010 08:26       2.55     17850.0  United Kingdom  
1  01-12-2010 08:26       3.39     17850.0  United Kingdom  
2  01-12-2010 08:26       2.75     17850.0  United Kingdom  
3  01-12-2010 08:26       3.39     17850.0  United Kingdom  
4  01-12-2010 08:26       3.39     17850.0  United Kingdom  
5  01-12-2010 08:26       7.65     17850.0  United Kingdom  


In [3]:
#if invoice has letter C then customer has cancelled order, so these are not considered
df[~df.InvoiceNo.str.contains("C")]
table=df.drop(columns="StockCode")
table=table.drop(columns="InvoiceDate")
table=table.drop(columns="Country")

In [4]:
table=table.drop(columns="InvoiceNo")
table=table.drop(columns="UnitPrice")
print(table.head(6))

                           Description  Quantity  CustomerID
0   WHITE HANGING HEART T-LIGHT HOLDER         6     17850.0
1                  WHITE METAL LANTERN         6     17850.0
2       CREAM CUPID HEARTS COAT HANGER         8     17850.0
3  KNITTED UNION FLAG HOT WATER BOTTLE         6     17850.0
4       RED WOOLLY HOTTIE WHITE HEART.         6     17850.0
5         SET 7 BABUSHKA NESTING BOXES         2     17850.0


In [5]:
def create_table_temp(table):
    table_temp = table.copy()
    table_temp['purchase_temp'] = 1
    return table_temp

In [6]:
table_temp = create_table_temp(table)

In [7]:
df_matrix = pd.pivot_table(table, values='Quantity', index='CustomerID', columns='Description')

In [8]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
d = df_matrix_norm.reset_index() 
d.index.names = ['Scaled_Purchase_Freq'] 
table_norm = pd.melt(d, id_vars=['CustomerID'], value_name='Scaled_Purchase_Freq').dropna()
print(table_norm.shape)
table_norm.head()

(268557, 3)


,CustomerID,Description,Scaled_Purchase_Freq
462,12937.0,4 PURPLE FLOCK DINNER CANDLES,0.000000
465,12940.0,4 PURPLE FLOCK DINNER CANDLES,0.000000
476,12953.0,4 PURPLE FLOCK DINNER CANDLES,0.454545
1192,13949.0,4 PURPLE FLOCK DINNER CANDLES,1.000000
1282,14071.0,4 PURPLE FLOCK DINNER CANDLES,0.000000


In [9]:
number = LabelEncoder();
table['Description']= number.fit_transform(table['Description'].astype('str'))
table_norm['Scaled_Purchase_Freq']= table_norm['Scaled_Purchase_Freq'].dropna()
table['CustomerID']= table['CustomerID'].dropna()
table['Description']= table['Description'].dropna()
table_diff= pd.DataFrame({"customer":table_norm["CustomerID"],
                           "item":table_norm["Description"],
                           "freq":table_norm["Scaled_Purchase_Freq"]})

In [10]:
results=table_diff.pivot_table(index='customer',columns='item',values='freq')
results

item,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TRELLIS COAT RACK,...,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
customer,,,,,,,,,,,,,,,,,,,,,
12346.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12347.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12348.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12349.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12350.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12352.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12353.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12354.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12355.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
allcustomers= results.values
customer12346= allcustomers[0]
denominator1= np.sqrt(sum([np.square(x) for x in customer12346]))
cosinesimilarity=[(12346,1)]
i=1
for customer in allcustomers[1:]:
    numerator= [x*y for x,y in zip(customer12346,customer)]
    denominator2= np.sqrt(sum([np.square(x) for x in customer]))
    costhetha= sum(numerator)/ (denominator1 * denominator2)
    cosinesimilarity.append((results.index[i],costhetha))
    i+=1
cosinesimilarity.sort(key=lambda x: x[1], reverse=True)
similar10customers= cosinesimilarity[0:10]
similar10customers

[(12346, 1),
 (12347.0, nan),
 (12348.0, nan),
 (12349.0, nan),
 (12350.0, nan),
 (12352.0, nan),
 (12353.0, nan),
 (12354.0, nan),
 (12355.0, nan),
 (12356.0, nan)]

In [12]:
top10customers= pd.DataFrame()
top10customers
for customers in similar10customers:
    top10customers= top10customers.append(results.loc[customers[0]])
top10customers['costheta']= [customer[1] for customer in similar10customers]

all_values = top10customers.values

top10customers

,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TRELLIS COAT RACK,...,ZINC METAL HEART DECORATION,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY,costheta
12346.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
12347.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12348.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12349.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12350.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12352.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12353.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12354.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12355.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12356.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
denominator= sum([x[1] for x in similar10customers])

inx = 0
values= []
for x in top10customers.loc[12346]:
    totalsum = 0
    if x == 0.0:
        for v in range(1,10):
            totalsum+= all_values[v-1][inx]*all_values[v-1][3570]
        top10customers.loc[12346][inx+1]= totalsum/denominator
    inx+=1
    
top10customers   

,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TRELLIS COAT RACK,...,ZINC METAL HEART DECORATION,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY,costheta
12346.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
12347.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12348.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12349.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12350.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12352.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12353.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12354.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12355.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12356.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
toprecommendations = []
for inx,x in enumerate(top10customers.loc[12346].values):
    if x > 0.0:
        toprecommendations.append((inx,x))
toprecommendations.sort(key=lambda x: x[1], reverse=True)

toprecommendations[:10]

[(3570, 1.0)]

In [15]:
colname=top10customers.columns[3570]
colname

'costheta'

In [16]:
#thus there are no recommendations for customer with customer id 12346

In [17]:
toprecommendations = []
for inx,x in enumerate(top10customers.loc[12355].values):
    if x > 0.0:
        toprecommendations.append((inx,x))
toprecommendations.sort(key=lambda x: x[1], reverse=True)

toprecommendations[:10]

[(1537, 0.3949579831932773),
 (1972, 0.3888888888888889),
 (3091, 0.38461538461538464),
 (2656, 0.34545454545454546),
 (2620, 0.3333333333333333),
 (2622, 0.30687830687830686),
 (949, 0.22499999999999998),
 (1373, 0.08),
 (2307, 0.08),
 (1998, 0.06167400881057269)]

In [18]:
#top movie recommendation for customer with customer id 12355
colname=top10customers.columns[1537]
colname

'ICE CREAM SUNDAE LIP GLOSS'